In [2]:
# Download dataset from
# url = "https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset/download?datasetVersionNumber=7"
# extract archive.zip

# from zipfile import ZipFile
  
# # loading the temp.zip and creating a zip object
# with ZipFile("archive.zip", 'r') as zObject:
  
#     # Extracting all the members of the zip 
#     # into a specific location.
#     zObject.extractall(path="archive")

import pandas as pd

df = pd.read_csv("archive/movies_metadata.csv")
df1 = df.head(10)
df1

/tmp/ipykernel_4868/3299589860.py:16: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("archive/movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
6,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0


In [3]:
import os
import json
import numpy as np
from numpy import nan
import requests
import base64
import time

errors_dict = {}
posters_imdbId_context_dict = {}
for index, row in df1.iterrows():
#     print(row['belongs_to_collection'])
    val = row['belongs_to_collection']
    if type(val) != str:
        continue
#     print(type(val))
#     print(val)
    try:
        val = val.replace("\'", "\"")
        dictData= json.loads(val)
        path = dictData.get('poster_path')
        imdb_id = dictData.get('id')
    except Exception as e:
        errors_dict[index] = e
#     print(path)
    url = "https://image.tmdb.org/t/p/w500/" + path
    img_data = requests.get(url).content
    os.makedirs("posters", exist_ok=True)
    img_location = 'posters/'+str(imdb_id)+'_poster.jpg'
    with open(img_location, 'wb') as handler:
        handler.write(img_data)
    with open(img_location, "rb") as img_file:
        my_string = base64.b64encode(img_file.read())
#     print(my_string.decode("utf-8"))
    hugging_face_url = "https://srddev-image-caption.hf.space/api/queue/push/"
    hugging_face_status_url = "https://srddev-image-caption.hf.space/api/queue/status/"
    
#     Needs to be removed, added to test for 1 case
#     if index > 1:
#         continue
    my_string = "data:image/jpeg;base64,"+my_string.decode("utf-8")
    payload = json.dumps({
      "fn_index": 0,
      "data": [
      my_string
    ],
      "action": "predict",
      "session_hash": "7p1oh5oyfh8"
    })
    headers = {
      'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/111.0',
      'Accept': '*/*',
      'Accept-Language': 'en-US,en;q=0.5',
      'Accept-Encoding': 'gzip, deflate, br',
      'Referer': 'https://srddev-image-caption.hf.space/?__theme=light',
      'Content-Type': 'application/json',
      'Origin': 'https://srddev-image-caption.hf.space',
      'Connection': 'keep-alive',
      'Cookie': 'session-space-cookie=acb1440f515e0f84f41ddb669ad094d1',
      'Sec-Fetch-Dest': 'empty',
      'Sec-Fetch-Mode': 'cors',
      'Sec-Fetch-Site': 'same-origin',
      'TE': 'trailers'
    }
    try:
        response = requests.request("POST", hugging_face_url, headers=headers, data=payload)
    #     print(response.text["hash"])
    #     payload = json.dumps({
    #       response.text
    #     })
        temp = {}
        while True:
            time.sleep(5) # average duration for hugging face to get the context of poster
            response1 = requests.request("POST", hugging_face_status_url, headers=headers, data=response)

#             print(response1.text)
            temp = json.loads(response1.text)
        #     print(temp)
            if temp['status'] == 'COMPLETE':
               break
#         print(temp['data']['data'][0])
        posters_imdbId_context_dict[imdb_id] = temp['data']['data'][0]
    except Exception as e:
        errors_dict[index] = e

{"status":"COMPLETE","data":{"data":["a woman in a hat and a cartoon character "],"duration":1.5078468322753906,"average_duration":18.39278984613933}}
a woman in a hat and a cartoon character 
{"status":"COMPLETE","data":{"data":["a collage of photos of people posing for a picture "],"duration":1.5967695713043213,"average_duration":18.322959471832622}}
a collage of photos of people posing for a picture 
{"status":"COMPLETE","data":{"data":["a bride and groom are posing for a picture "],"duration":1.4048559665679932,"average_duration":18.288076784192903}}
a bride and groom are posing for a picture 
{"status":"PENDING","data":null}
{"status":"PENDING","data":null}
{"status":"PENDING","data":null}
{"status":"PENDING","data":null}
{"status":"COMPLETE","data":{"data":["a black and white photo of a man wearing a bow tie "],"duration":26.856385231018066,"average_duration":18.30570704848678}}
a black and white photo of a man wearing a bow tie 


In [4]:
posters_imdbId_context_dict

{10194: 'a woman in a hat and a cartoon character ',
 119050: 'a collage of photos of people posing for a picture ',
 96871: 'a bride and groom are posing for a picture ',
 645: 'a black and white photo of a man wearing a bow tie '}

In [5]:
import csv


with open('dict.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in posters_imdbId_context_dict.items():
       writer.writerow([key, value])